In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
os.chdir("/home/yw699/codes/LLM-Hallu")
sys.path.append(os.path.abspath("src"))

In [4]:
from uncertainty_measures import *

In [6]:
entailment_model = EntailmentDeberta()

context = "The weather is good."
responses = [
    "The weather is good and I like you.",
    "It is sunny outside.",
    "The weather is terrible today.",
    "The weather is good."
]

# Step 1: 检查上下文是否支持每个生成文本
print("\nChecking implications between context and responses:")
for response in responses:
    result = entailment_model.check_implication(context, response)
    print(f"Context -> '{response}' | Implication Result: {result} (2: entailment, 1: neutral, 0: contradiction)")

# Step 2: 按语义分组生成文本
print("\nGrouping responses into semantic clusters:")
semantic_ids = get_semantic_ids(responses, model=entailment_model)
print(f"Semantic IDs: {semantic_ids}")

# Step 3: 计算语义聚类的熵
cluster_entropy = cluster_assignment_entropy(semantic_ids)
print(f"\nCluster Assignment Entropy: {cluster_entropy:.4f}")

# Step 4: 模拟生成概率并计算预测熵
log_probs = [-0.5, -0.7, -1.2, -0.6]  # 假设生成的对数概率
regular_entropy = predictive_entropy(log_probs)
print(f"\nPredictive Entropy (Regular): {regular_entropy:.4f}")

tokenizer_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/952 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.45M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.77G [00:00<?, ?B/s]


Checking implications between context and responses:
Context -> 'The weather is good and I like you.' | Implication Result: 1 (2: entailment, 1: neutral, 0: contradiction)
Context -> 'It is sunny outside.' | Implication Result: 1 (2: entailment, 1: neutral, 0: contradiction)
Context -> 'The weather is terrible today.' | Implication Result: 0 (2: entailment, 1: neutral, 0: contradiction)
Context -> 'The weather is good.' | Implication Result: 2 (2: entailment, 1: neutral, 0: contradiction)

Grouping responses into semantic clusters:
Semantic IDs: [0, 1, 2, 0]

Cluster Assignment Entropy: 1.0397

Predictive Entropy (Regular): 0.7500


## 读取配置文件

In [ ]:
import yaml
with open("configs/experiment_config1.yaml", "r") as file:
    config = yaml.safe_load(file)

## Dataset加载与格式化

In [ ]:
from utils import *
from dataset import Dataset
dataset_loader = Dataset(config)
train_dataset, validation_dataset = dataset_loader.load_data()
print(validation_dataset[2])  

if not isinstance(train_dataset, list):
        logging.info('Train dataset: %s', train_dataset)

answerable_indices, unanswerable_indices = split_dataset(train_dataset)


if config["dataset"]['answerable_only']:
        unanswerable_indices = []
        val_answerable, val_unanswerable = split_dataset(validation_dataset)
        del val_unanswerable
        validation_dataset = [validation_dataset[i] for i in val_answerable]

## Prompt Engineer

In [ ]:
from prompt_engineer import PromptGenerator

promptgenerator = PromptGenerator(config,validation_dataset)

In [ ]:
prompt = promptgenerator.generate_template_prompt_by_id(1)
print(prompt)
print(len(promptgenerator.get_unused_indices()))


In [ ]:
prompts = promptgenerator.generate_template_prompts_by_count(2)
print(prompt)
print(len(promptgenerator.get_unused_indices()))

In [ ]:
import random
indices = random.sample(promptgenerator.get_unused_indices(),2)
print(indices)
prompts =promptgenerator.generate_template_prompts_from_indices(indices)
len(promptgenerator.get_unused_indices())

In [ ]:
few_shot_prompt = promptgenerator.construct_fewshot_prompt_from_indices()
print(few_shot_prompt)
print(len(promptgenerator.get_unused_indices()))

## model

In [ ]:
from models import HuggingfaceModel
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
huggingface_model = HuggingfaceModel(config)

In [ ]:
#temperature=1.0,return_full=False
output_text = huggingface_model.predict(prompt)
print("Generated Text > \n", output_text)

In [ ]:
import math

user_tag = "USER:"
assistant_tag = "ASSISTANT:"
input = "What is the capital of France?"
v = huggingface_model.get_p_true(input,"Washington")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Paris")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")


v = huggingface_model.get_p_true(input,"Beijing")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

v = huggingface_model.get_p_true(input,"\n")
probability = math.exp(v)
print(f"Probability: {probability:.4f}")

In [ ]:
input_text = prompts[1]

output_text,log = huggingface_model.predict(input_text)
print("<Generated Text>:", output_text)
print(log)